In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (6800 Fisher Island Dr #6812, Miami Beach, FL ...
2     (10201 Collins Ave #1807w, Bal Harbour, FL 331...
3     (450 Alton Rd APT 2301, Miami Beach, FL 33139,...
4     (3581 E Glencoe St #309, Miami, FL 33133, USA,...
5     (18975 Collins Ave #403, Sunny Isles Beach, FL...
6     (4701 Meridian Ave #319, Miami Beach, FL 33140...
7     (50 South Pointe Dr APT 1104, Miami Beach, FL ...
8     (1300 Brickell Bay Dr APT 4302, Miami, FL 3313...
9     (791 Crandon Blvd APT 1501, Key Biscayne, FL 3...
10    (5660 Collins Ave #20d, Miami Beach, FL 33140,...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Palazzo Della Luna\r6800 Fisher Island Dr 6812...,3/25/2021,2/22/2022,290,"$21,250,000.00",9361,"$4,810.96",Dora Puig,Dora Puig,Palazzo Della Luna 6800 Fisher Island Dr 6812 ...,Palazzo Della Luna,6800 Fisher Island Dr 6812 Miami Beach,"(6800 Fisher Island Dr #6812, Miami Beach, FL ...","(25.7623668, -80.14735929999999, 0.0)",25.762367,-80.147359,0.0
2,Oceana Bal Harbour\r10201 Collins Ave 1807W\rB...,12/30/2021,2/23/2022,18,"$4,200,000.00",2226,"$1,886.79",Matias Derito,Matias Derito,Oceana Bal Harbour 10201 Collins Ave 1807W Bal...,Oceana Bal Harbour,10201 Collins Ave 1807W Bal Harbour,"(10201 Collins Ave #1807w, Bal Harbour, FL 331...","(25.8951677, -80.12315, 0.0)",25.895168,-80.123150,0.0
3,Icon South Beach\r450 Alton Rd 2301\rMiami Beach,1/6/2022,2/25/2022,18,"$3,800,000.00",2145,"$1,771.56",Timothy Hartmann,Christian Dreyfuss PA,Icon South Beach 450 Alton Rd 2301 Miami Beach,Icon South Beach,450 Alton Rd 2301 Miami Beach,"(450 Alton Rd APT 2301, Miami Beach, FL 33139,...","(25.7719135, -80.1380353, 0.0)",25.771914,-80.138035,0.0
4,The Fairchild Condo\r3581 E Glencoe St 309\rMiami,11/30/2021,2/24/2022,5,"$3,360,000.00",2984,"$1,307.90",Patrick Vysata,Ivan Chorney,The Fairchild Condo 3581 E Glencoe St 309 Miami,The Fairchild Condo,3581 E Glencoe St 309 Miami,"(3581 E Glencoe St #309, Miami, FL 33133, USA,...","(25.7400814, -80.2173492, 0.0)",25.740081,-80.217349,0.0
5,Residences by Armani Casa\r18975 Collins Ave 4...,7/17/2021,2/22/2022,194,"$3,350,000.00",2271,"$1,475.12",Marcelo Violland,Katia Reisler,Residences by Armani Casa 18975 Collins Ave 40...,Residences by Armani Casa,18975 Collins Ave 403 Sunny Isles Beach,"(18975 Collins Ave #403, Sunny Isles Beach, FL...","(25.9519943, -80.11997, 0.0)",25.951994,-80.119970,0.0
6,Ritz Carlton Residences\r4701 N Meridian Ave 3...,5/16/2021,2/22/2022,249,"$3,030,000.00",3081,$983.45,Tomas Hoffmann,Stephan Burke,Ritz Carlton Residences 4701 N Meridian Ave 31...,Ritz Carlton Residences,4701 N Meridian Ave 319 Miami Beach,"(4701 Meridian Ave #319, Miami Beach, FL 33140...","(25.8227618, -80.13216160000002, 0.0)",25.822762,-80.132162,0.0
7,Continuum North\r50 S Pointe Dr 1104\rMiami Beach,12/27/2021,2/25/2022,13,"$2,710,000.00",1443,"$1,878.03",Juan Alvarez,Michelle Gunter,Continuum North 50 S Pointe Dr 1104 Miami Beach,Continuum North,50 S Pointe Dr 1104 Miami Beach,"(50 South Pointe Dr APT 1104, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0
8,Brickellhouse Condo\r1300 Brickell Bay Dr 4302...,9/16/2021,2/22/2022,121,"$2,550,000.00",3273,$779.10,Rodrigo Kirilauscas,Kiel Pollitt,Brickellhouse Condo 1300 Brickell Bay Dr 4302 ...,Brickellhouse Condo,1300 Brickell Bay Dr 4302 Miami,"(1300 Brickell Bay Dr APT 4302, Miami, FL 3313...","(25.7603908, -80.19034649999999, 0.0)",25.760391,-80.190346,0.0
9,Ocean Tower Two Condo\r791 Crandon Blvd 1501\r...,10/1/2021,2/22/2022,103,"$2,485,000.00",2174,"$1,143.05",Jacqueline Gallardo,Lourdes Lorenzo-Luaces,Ocean Tower Two Condo 791 Crandon Blvd 1501 Ke...,Ocean Tower Two Condo,791 Crandon Blvd 1501 Key Biscayne,"(791 Crandon Blvd APT 1501, Key Biscayne, FL 3...","(25.6883367, -80.1579345, 0.0)",25.688337,-80.157934,0.0
10,Fifty Six-Sixty\r5660 Collins Ave 20D\rMiami B...,9/21/2021,2/23/2022,122,"$2,195,000.00",3068,$819.64,Theresa Maieli,Ken Rubman LLC,Fifty Six-Sixty 5660 Collins Ave 20D Miami Beach,Fifty Six-Sixty,5660 Collins Ave 20D Miami Beach,"(5660 Collins Ave #20d, Miami Beach, FL 33140,...","(25.8374932, -80.1213544, 0.0)",25.837493,-80.121354,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [11]:
df.at[1,'Agent']=('Dora Puig, Carl Gambino')

In [12]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [13]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 20th - February 26th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [14]:
m.save('index.html')